The goal is to given a folder with documents, return a folder with different csvs of common data from the documents.
and 
Evaluation  

*** !!!! This contains code from https://github.com/microsoft/table-transformer
the areas where code has been taken and modified are labeled clearly. This includes the 
factor2dmass method which finds the most similar substructure for evaluation purposes

This notebook relies on cloning the repo as running DETR+TATR for evaluation utilizes their inference pipeline

In [ ]:
import os
import sys
import fitz
from PIL import Image
import json
import re

In [ ]:
sys.path.append('../table-transformer/')
sys.path.append('C:\\Users\\13109\\Documents\\college\\fall2023\\8803\\project\\table-transformer\\src')
sys.path.append('C:\\Users\\13109\\Documents\\college\\fall2023\\8803\\project\\table-transformer\\detr')

In [ ]:
from src.inference import TableExtractionPipeline,visualize_detected_tables,visualize_cells

# Create inference pipeline
detection_config_path = 'src/detection_config.json'
detection_model_path = "src/pubtables1m_detection_detr_r18.pth" #'../pubtables1m_detection_detr_r18.pth'
detection_device = 'cpu'
structure_config_path = 'src/structure_config.json'
structure_model_path = 'TATR-v1.1-Fin-msft.pth'
structure_device = 'cpu'
pipe = TableExtractionPipeline(det_config_path=detection_config_path, det_model_path=detection_model_path, det_device=detection_device, str_config_path=structure_config_path, str_model_path=structure_model_path, str_device=structure_device)

In [ ]:
def get_page_words(page):
    words = []

    for text_word in page.get_text_words():
        word = {'bbox': list(text_word[:4]),
                'text': text_word[4],
                'block_num': text_word[5],
                'line_num': text_word[6],
                'span_num': text_word[7],
                'flags': 0}
        words.append(word)

    return words

In [ ]:
#Recognize tables from document images ## used for runtime measuring as well
documents_path = '../documents_1210'
for filename in list(os.listdir(documents_path)):
    output_dir = f"../detrtatr_csvs_1210"
    if not filename.endswith(".pdf"):
        continue
    doc = fitz.open(os.path.join(documents_path, filename))
    page = doc.load_page(0)
    words = get_page_words(page) # or use FinTabNet.c // image table words but scaled to whole page from bounding box coords
    pix = page.get_pixmap()
    output = f"{documents_path}/document_images/{filename.replace('.pdf','' )}.jpg"
    pix.save(output)
    pix.pil_save(output)
    img = Image.open(output)
    img_path = output
    
    print(img_path)
    extracted_tables = pipe.extract(img, tokens=words, out_objects=True, out_cells=True,
                out_html=False, out_csv=True)
    for table_idx, extracted_table in enumerate(extracted_tables):
        for key, val in extracted_table.items():
            if key == 'csv':
                csv_filename = os.path.join(output_dir, filename.replace(".pdf", ".csv"))
                if val != None and len(val) != 0 and val[0] != None:
                    with open(csv_filename, 'w', encoding = 'utf-8') as f:
                        f.write(val[0])

### Evaluation

In [ ]:
$env:PYTHONPATH = "C:\Users\13109\Documents\college\fall2023\8803\project\table-transformer\src;" + $env:PYTHONPATH
$env:PYTHONPATH = "C:\Users\13109\Documents\college\fall2023\8803\project\table-transformer\detr;" + $env:PYTHONPATH

This is only to save csvs for ground truth. table_transformer main code was modified to save during the benchmark evaluation execution

In [ ]:
!python ./src/main.py --mode eval --data_type structure --config_file 'src/structure_config.json' --data_root_dir "../FinTabNet-r.c/FinTabNet.c_Image_Structure_PASCAL_VOC" --model_load_path TATR-v1.1-Fin-msft.pth --table_words_dir "../FinTabNet-r.c/FinTabNet.c_Image_Table_Words_JSON" --test_max_size 50 --device cpu

In [ ]:
"..\src\pubtables1m_detection_detr_r18.pth"
"..\src\detection_config.json"
"..\documents_1210\document_images"

Inference pipeline to test token and table extraction 

In [ ]:
!python ./src/inference.py --mode extract --detection_config_path ".\src\detection_config.json" --detection_model_path ".\src\pubtables1m_detection_detr_r18.pth" --detection_device cpu --structure_config_path 'src/structure_config.json' --structure_model_path TATR-v1.1-Fin-msft.pth --structure_device cpu --image_dir "..\documents_1210\document_images" --words_dir "..\\FinTabNet-r.c\\FinTabNet.c_Image_Table_Words_JSON" --out_dir "../inference_1210/" -c --crop_padding 20

## Evaluation

In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from src.grits import factored_2dmss, iou
import itertools
from difflib import SequenceMatcher

In [61]:
list_of_ground_truth = set()
for csvs in os.listdir('../true_csvs/'):
    if csvs.endswith('.csv'):
        table_name = csvs.split('/')[-1]
        table_name = table_name.split('_')[:4]
        table_name = "_".join(table_name)
        list_of_ground_truth.add(table_name)
print(len(list_of_ground_truth))

8000


In [ ]:
for cam in os.listdir('../camelot_csv/camelot_evaluation_csv'):
    #rename without camelot_eval
    if cam.endswith('.csv'):
        table_name = cam.split('/')[-1]
        table_name = table_name.split('_')[:4]
        table_name = "_".join(table_name)
        os.rename(f'../camelot_csv/camelot_evaluation_csv/{cam}', f'../camelot_csv/camelot_evaluation_csv/{table_name}.csv')

In [ ]:
ground_truth_path = ".//true_csvs"
detrtatr_path = ".//detrtatr_csvs"
camelot_path = ".//camelot_csv/camelot_evaluation_csv"
combined_path = ".//combined_csvs"

In [ ]:
def clean(flattened):
    # remove punctuation
    flattened = flattened.replace(',', '')
    flattened = flattened.replace('.', '')
    # remove long whitespace
    flattened = flattened.replace('\n', ' ')
    flattened = flattened.replace('\t', ' ')
    flattened = flattened.replace('   ', ' ')
    return flattened 


In [ ]:
def align_2d_outer(shape1, shape2, rewards):
    num_rows, num_cols = min(shape1[0], shape2[0]), min(shape1[1], shape2[1])
    return list(range(num_rows)), list(range(num_cols)), 0

def compute_fscore(positive_match_score, num_true, num_pos):
    precision = positive_match_score / num_true if num_true > 0 else 0
    recall = positive_match_score / num_pos if num_pos > 0 else 0
    fscore = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return fscore, precision, recall

def lcs_similarity(string1, string2):
    string1 =str(string1); string2 = str(string2)
    if len(string1) == 0 and len(string2) == 0:
        return 1
    s = SequenceMatcher(None, string1, string2)
    lcs = ''.join([string1[block.a:(block.a + block.size)] for block in s.get_matching_blocks()])
    return 2*len(lcs)/(len(string1)+len(string2))

def factored_2dmss(df1, df2, reward_function):
    # Convert DataFrames to NumPy arrays
    """
    based on code from eval.py in 
    """
    true_cell_grid = df1.to_numpy()
    pred_cell_grid = df2.to_numpy()

    pre_computed_rewards = {}
    transpose_rewards = {}
    for trow, tcol, prow, pcol in itertools.product(range(true_cell_grid.shape[0]),
                                                    range(true_cell_grid.shape[1]),
                                                    range(pred_cell_grid.shape[0]),
                                                    range(pred_cell_grid.shape[1])):

        reward = reward_function(true_cell_grid[trow, tcol], pred_cell_grid[prow, pcol])

        pre_computed_rewards[(trow, tcol, prow, pcol)] = reward 
        transpose_rewards[(tcol, trow, pcol, prow)] = reward

    num_pos = pred_cell_grid.shape[0] * pred_cell_grid.shape[1]
    num_true = true_cell_grid.shape[0] * true_cell_grid.shape[1]

    # Align rows and columns
    true_row_nums, pred_row_nums, row_pos_match_score = align_2d_outer(true_cell_grid.shape[:2],
                                                                       pred_cell_grid.shape[:2],
                                                                       pre_computed_rewards)
    true_column_nums, pred_column_nums, col_pos_match_score = align_2d_outer(true_cell_grid.shape[:2][::-1],
                                                                             pred_cell_grid.shape[:2][::-1],
                                                                             transpose_rewards)

    pos_match_score_upper_bound =  min(row_pos_match_score, col_pos_match_score)
    upper_bound_score, _, _ = compute_fscore(pos_match_score_upper_bound, num_pos, num_true)

    # Calculate the positive match score
    positive_match_score = 0
    for true_row_num, pred_row_num in zip(true_row_nums, pred_row_nums):
        for true_column_num, pred_column_num in zip(true_column_nums, pred_column_nums):
            positive_match_score += pre_computed_rewards[(true_row_num, true_column_num, pred_row_num, pred_column_num)]

    # Compute the F-score, precision, and recall
    fscore, precision, recall = compute_fscore(positive_match_score, num_true, num_pos)
    
    return fscore, precision, recall, upper_bound_score


In [ ]:
def grits_2dmss(ground_truth_df, model_output_df):
    # put to np arrays
    ground_truth_df = ground_truth_df.to_numpy()
    model_output_df = model_output_df.to_numpy()
    # run grits
    print(model_output_df.shape, ground_truth_df.shape)

    return factored_2dmss(ground_truth_df, model_output_df, iou)


def jaccard_similarity(set1, set2):
    # Calculate the Jaccard Similarity between two sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if len(union) != 0 else 1

def shape_accuracy(ground_truth_shape, model_output_shape):
    # Calculate the percentage match for rows and columns
    row_accuracy = min(ground_truth_shape[0], model_output_shape[0]) / max(ground_truth_shape[0], model_output_shape[0])
    col_accuracy = min(ground_truth_shape[1], model_output_shape[1]) / max(ground_truth_shape[1], model_output_shape[1])

    # print('row accuracy: ',  str(row_accuracy))
    # print('col accuracy: ', str(col_accuracy))
    # Calculate overall structural accuracy
    return (row_accuracy + col_accuracy) / 2 

def compare_contents(ground_truth_df, model_output_df):
    # Flatten the DataFrames into lists or sets
    ground_truth_content = ' '.join(ground_truth_df.astype(str).values.flatten()) + ' '.join(ground_truth_df.columns)
    model_output_content = ' '.join(model_output_df.astype(str).values.flatten()) + ' '.join(model_output_df.columns)
    ground_truth_content = clean(ground_truth_content)
    model_output_content = clean(model_output_content)

    vectorizer = CountVectorizer().fit_transform([ground_truth_content, model_output_content])
    vectors = vectorizer.toarray()

    return cosine_similarity(vectors)[0][1] * 100


def compare_tables(ground_truth_csv, model_output_csv):
    column_similarity = jaccard_similarity(set(ground_truth_csv.columns), set(model_output_csv.columns))
    shape_acc = shape_accuracy(ground_truth_csv.shape, model_output_csv.shape)
    content_accuracy = compare_contents(ground_truth_csv, model_output_csv)
    jac = jaccard_similarity(set(ground_truth_csv.values.flatten()), set(model_output_csv.values.flatten()))
    grits = factored_2dmss(ground_truth_csv, model_output_csv, lcs_similarity)

    return shape_acc * 100, content_accuracy,  column_similarity*100, jac, grits

In [ ]:
ground_truth_path = ".//true_csvs"
detrtatr_path = ".//detrtatr_csvs"
camelot_path = ".//camelot_csv/camelot_evaluation_csv"
combined_path = ".//combined_csvs"

In [ ]:
big = []
test_acc = [0,0,0,0, 0]
combined_acc = [0,0,0,0, 0]; combined_count = 0
camelot_acc = [0,0,0,0, 0]
test_grits = []
combined_grits = []
camelot_grits = []
count = 0
for doc in os.listdir(combined_path):
    doc_name = doc.split('_')[:4]
    doc_name = "_".join(doc_name)
    if doc.endswith('.csv'):
            true_csv_ = pd.read_csv(f'../{ground_truth_path}/{doc_name.strip(".csv")}_true_table_0.csv')
            detrtatr_csvs_ = f'../{detrtatr_path}/{doc_name}'
            combined_csvs_ = f'../{combined_csvs_}/{doc_name}'
            camelot_csvs_ = f'{camelot_csvs_}/{doc_name}'
            for id, table in enumerate([detrtatr_csvs_, combined_csvs_, camelot_csvs_]):
                try:
                    table_ = pd.read_csv(table)
                except:
                    print("error", table)
                    continue # Track to remove errors in filepaths
                print()
                print("Accuracy:")
                results = (compare_tables(true_csv_, table_))
                print(id, results)
                if id == 0:
                    test_acc = [acc+r for acc, r in zip(test_acc[:-1], results[:-1])]
                    test_grits.append(results[-1])
                elif id == 1:
                    combined_acc = [acc+r for acc, r in zip(combined_acc[:-1], results[:-1])]
                    combined_grits.append(results[-1])
                else:
                    camelot_acc = [acc+r for acc, r in zip(camelot_acc[:-1], results[:-1])]
                    camelot_grits.append(results[-1])


In [71]:
for i,  grits in enumerate([test_grits, combined_grits, camelot_grits]):
    print(['Detr+Tatr','Combined','Camelot'][i])
    temp = []
    for col in range(4):
        temp.append(sum([g[col] for g in grits]))
    for label, score in zip(['Average F-score', 'Average Precision', 'Average Recall'], temp):
        score = 
        print(label,' : ', round(score/ len(list_of_ground_truth), 3))
    print()


Detr+Tatr
Average F-score :  0.684
Average Precision : 0.721
Average Recall : 0.651

Combined
Average F-score : 0.692
Average Precision : 0.679 
Average Recall : 0.705

Camelot
Average F-score : 0.766
Average Precision : 0.485
Average Recall : 0.595


get randomized csvs from eval + detrtatr run

In [ ]:
# loop through all csvs in test_csvs_1210, 
pdfs = "../table-transformer/fintabnet/fintabnet/pdf/"
save_path = "../documents"
list_of_ground_truth = set()
for csv_ in os.listdir('../test_csvs/'):
    if csv_.endswith('.csv'):
        csv_name = csv_.split('_')[:4]
        csv_name = "_".join(csv_name)
        list_of_ground_truth.add(csv_name)
        #
        company, year, quarter, page = csv_name.split('_')
        print(company, year, quarter, page)
        path = pdfs + company + '/' + year + '/' + quarter + '_' + page + '.pdf'
        print(path)
        print(csv_name)
        doc = fitz.open(path)
        # save pdf in save_path
        print(save_path +'/' + csv_name + '.pdf')
        doc.save(save_path +'/' + csv_name + '.pdf')